# Factor Analysis

This notebook is based on/extending an article posted originally on [DataCamp](https://www.datacamp.com/community/tutorials/introduction-factor-analysis).  The original article is aging, and its example code hasn't been updated to reflect changes in the `factor_analyzer` package used.  The info and discussion from article still a good resource.

The analysis will be focused on the `BFI` dataset; more information on this data can be found [here](https://www.personality-project.org/r/html/bfi.html).

In [0]:
# %reload_ext nb_black

In [0]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA, FastICA

# !pip install factor_analyzer
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

data_url = "https://vincentarelbundock.github.io/Rdatasets/csv/psych/bfi.csv"
bfi = pd.read_csv(data_url, index_col=0)

bfi.head()

Our features of interest for this analysis will be the responses to the questions.  These are all the features excluding the subject id and demographic information.

In [0]:
# fmt: off
feat_names = ['A1', 'A2', 'A3', 'A4', 'A5', 
              'C1', 'C2', 'C3', 'C4', 'C5',
              'E1', 'E2', 'E3', 'E4', 'E5', 
              'N1', 'N2', 'N3', 'N4', 'N5', 
              'O1', 'O2', 'O3', 'O4', 'O5']
# fmt: on

X = bfi[feat_names]

print("Top missing value features for X:")
print(X.isnull().mean().sort_values(ascending=False).head())
print(f"\nX shape: {X.shape}")

Dropping our missing values doesn't impact too much in light of our missing value rate and sample size.  We'll just drop them without too much thought in this demo.

In [0]:
X = X.dropna()
print(f"X shape: {X.shape}")

If we wanted to make predictions with this data using a linear model (or something similiar) we'd want to check for multicollinearity.  Let's make a heatmap and see if we spot some potentially assumption breaking correlations.

From the heatmap, we see some highly correlated blocks.  The most obvious examples obvious examples are the `N*` features and the `A*` features.

In [0]:
sns.heatmap(X.corr())
plt.show()

If we wanted to proceed with building a model, we might want to do some dimension reduction first.  Let's use factor analysis through the `factor_analyzer` package.  This package implements the commonly used `varimax` rotation [unlike `sklearn`](https://github.com/scikit-learn/scikit-learn/issues/2688) (`varimax` appears to be [under development still for `sklearn`](https://github.com/scikit-learn/scikit-learn/pull/11064) as of writing this).

Before running a factor analysis, we need to do some "adequacy tests".  If these tests fail, we should not run a factor analysis.

> * Bartlett’s test of sphericity checks whether or not the observed variables intercorrelate at all using the observed correlation matrix against the identity matrix. If the test found statistically insignificant, you should not employ a factor analysis.
> * Kaiser-Meyer-Olkin (KMO) Test measures the suitability of data for factor analysis. It determines the adequacy for each observed variable and for the complete model. KMO estimates the proportion of variance among all the observed variable. Lower proportion id more suitable for factor analysis. KMO values range between 0 and 1. Value of KMO less than 0.6 is considered inadequate.

In [0]:
_, p_value = calculate_bartlett_sphericity(X)
print(f"* Passed Bartlett adequacy test?\n\t{p_value < 0.05} (p = {p_value:.4f})")

_, kmo = calculate_kmo(X)
print(f"* Passed Kaiser-Meyer-Olkin adequacy test?\n\t{kmo >= 0.6} (kmo = {kmo:.4f})")

Cool, we passed our tests. Lets get to factor analyzing.

But how many factors should we choose?  One way to do this is with the [Kaiser Criterion](https://en.wikipedia.org/wiki/Factor_analysis#Older_methods), which is nice due to its simplicity.  We calclulate some eigenvalues and count how many are above 1.  Other [more 'modern' methods](https://en.wikipedia.org/wiki/Factor_analysis#Modern_criteria) are harder to find implementations for.

In [0]:
# This will throw a FutureWarning
# (factor_analyzer version at time of writing this is 0.3.2)
fa = FactorAnalyzer(n_factors=25, rotation="varimax")
_ = fa.fit_transform(X)
ev, _ = fa.get_eigenvalues()

`matplotlib` practice!  Plot the eigenvalues (stored in `ev`)

* [ ] your x axis should be the range [1, # of eigenvalues]
* [ ] your y axis should be the eigenvalue
* [ ] plot a horizontal line at `y=1`
  * [ ] make the line dashed
  * [ ] make the line black
* [ ] for eigenvalues > 1, make the scatter marker a different color
* [ ] give the plot meaningful axis labels and a title

From our candidate 25 factors, 6 meet the cutoff value of 1.  The 6th is pretty close to our cutoff, but we'll keep it in the set until it's proven not to be useful.

In [8]:
# This will throw a FutureWarning
# (factor_analyzer version at time of writing this is 0.3.2)
fa = FactorAnalyzer(n_factors=6, rotation="varimax")
X_transformed = fa.fit_transform(X)

print(X.shape)
print(X_transformed.shape)

(2436, 25)
(2436, 6)


We successfully trimmed down our feature set to 6... so what?  We could do that with PCA or a lot of other methods. 

Let's look at the factor `loadings_` to see how the original features map to our reduced factors.  We'll use some `pandas` styling to highlight the magnitude of each feature's contribution to the factor (note that this is the *magnitude* aka absolute value).

In [0]:
# Use the loadings_ attribute to build a dataframe 
# Make the dataframe row names the orignal feature names


# Display the loadings with each cell colored by the value

Our 6th factor isn't adding too much since our first 5 are covering pretty much all the questions.  This factor also doesn't have the benefit of being interpretable like the others.  Recall, it was also borderline with our eigenvalue > 1 rule-of-thumb; let's drop it and move on.

From our factor analysis we reduced our dimensions and still kept a high level of intpretability in the terms of this dataset.  We can look at another correlation heatmap to see if our data is now green lit for a model that assumes no multicollinearity.

Let's also quickly throw `PCA` at our data to contrast the 2 methods.

From our output we see that unlike factor analysis, the loadings don't offer an easy interpretation.

In [0]:
def pca_loadings(pca):
    return pca.components_.T * np.sqrt(pca.explained_variance_)

In [0]:
# Perform pca with 5 components and create the same output as for FA
pca = 

loadings = pca_loadings(pca)

Let's also throw ICA in the mix.

In [0]:
ica = FastICA(n_components=5)
nba_ics = ica.fit_transform(X)

loadings = pd.DataFrame(ica.mixing_)
loadings.index = X.columns
loadings.columns = [f"ind_component_{i}" for i in range(loadings.shape[1])]

# # Sort by absolute value of column
# sort_order = loadings.abs().sort_values("ind_component_0", ascending=False).index
# loadings = loadings.reindex(sort_order)

print("Component loadings absolute values (colored by magnitude)")
loadings.abs().style.background_gradient(axis=None)